<a href="https://colab.research.google.com/github/viliturpeinen/Colabs/blob/main/TEHTAVA1_KOE27112025_PAATOKSENTEKOMALLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pulp --quiet

import pulp

# Päätösmuuttujat: x_A, x_B, x_C = viikottain valmistettavat kpl

x = pulp.LpVariable.dicts("Kone", ["A", "B", "C"], lowBound=0, cat='Continuous')

# Parametrit

profit = {"A": 250.5, "B": 185.5, "C": 210}
time = {
"Osien_muokkaus": {"A":4, "B":2.5, "C":3.3, "Max":4700},
"Kokoaminen": {"A":16.5, "B":8, "C":14.5, "Max":15500},
"Viimeistely": {"A":6.5, "B":5, "C":8.5, "Max":8400},
"Tarkastus_pakkaus": {"A":2, "B":2, "C":2, "Max":2300},
}
max_sales = {"A":75, "B":135, "C":120}

# Optimointimalli

model = pulp.LpProblem("Tuotanto_optimointi", pulp.LpMaximize)

# Tavoitefunktion määrittely (voiton maksimointi)

model += pulp.lpSum([profit[i]*x[i] for i in ["A","B","C"]]), "Kokonaisvoitto"

# Rajoitteet: työvaiheet

for phase in ["Osien_muokkaus","Kokoaminen","Viimeistely","Tarkastus_pakkaus"]:
    model += pulp.lpSum([time[phase][i]*x[i] for i in ["A","B","C"]]) <= time[phase]["Max"], phase+"_rajoite"

# Rajoitteet: myynti/linja

for i in ["A","B","C"]:
    model += x[i] <= max_sales[i], i+"_myyntirajoite"

# Ratkaistaan ongelma

model.solve()

# Tulostetaan tulokset

print("Status:", pulp.LpStatus[model.status])
for i in ["A","B","C"]:
    print(f"{i} linjan tuotanto: {x[i].varValue:.2f} kpl/viikko")
print(f"Maksettu voitto yhteensä: {pulp.value(model.objective):.2f} €")

# --- Shadow price (varjohinta) työvaiheille ---

print("\nDual-arvot (varjohinnat) työvaiheille [€/tunti]:")
shadow_prices = {}
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.pi:.2f}")
    shadow_prices[name] = constraint.pi

# Arvio kokoamisen lisäresurssin maksukyvystä

max_extra_cost = shadow_prices.get("Kokoaminen_rajoite",0)
print(f"\nKokoamiseen lisäresurssin maksimikustannus €/tunti, jotta se kannattaa: {max_extra_cost:.2f} €/tunti")

Status: Optimal
A linjan tuotanto: 75.00 kpl/viikko
B linjan tuotanto: 135.00 kpl/viikko
C linjan tuotanto: 120.00 kpl/viikko
Maksettu voitto yhteensä: 69030.00 €

Dual-arvot (varjohinnat) työvaiheille [€/tunti]:
Osien_muokkaus_rajoite: -0.00
Kokoaminen_rajoite: -0.00
Viimeistely_rajoite: -0.00
Tarkastus_pakkaus_rajoite: -0.00
A_myyntirajoite: 250.50
B_myyntirajoite: 185.50
C_myyntirajoite: 210.00

Kokoamiseen lisäresurssin maksimikustannus €/tunti, jotta se kannattaa: -0.00 €/tunti
